In [1]:
import os
import glob
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import shutil
import cv2

In [2]:
home_path = os.getcwd()
image_path = os.path.join(home_path,'images')
data_path = os.path.join(home_path,'data')

# Step 1: convert xml files to csv

In [4]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        filename = xml_file.split("/")[-1]
        filename = ".".join(filename.split(".")[:-1])
        filename = filename + '.jpg'
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            name = member[0].text
            if(name == 'cars' or name == 'caar'):
                name = 'car'
            elif(name == 'ayto' or name =='autor' or name == 'autoclave'):
                name = 'auto'
            elif(name == 'moto' or name == 'motor' or name == 'motorbikes' or name == 'motorbike cart' or name == 'motobike'):
                name = 'motorbike'
            elif(name == 'vehicle' or name == 'trucck' or name == 'truk'):
                name = 'truck'
            elif(name == 'truck crane' or name == 'tractor'):
                name = 'tractor'
            value = (filename,
            int(root.find('size')[0].text),
            int(root.find('size')[1].text),
            name,
            int(member[4][0].text),
            int(member[4][1].text),
            int(member[4][2].text),
            int(member[4][3].text)
            )
            if value[3] in ['car','truck','bus','tractor','motorbike','auto','animal','person']:
                xml_list.append(value)

    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [7]:
xml_df = xml_to_csv(image_path)
xml_df.to_csv(data_path+'/labels.csv', index=None)
print('Successfully converted xml to csv.')
print(xml_df.info())

Successfully converted xml to csv.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12664 entries, 0 to 12663
Data columns (total 8 columns):
filename    12664 non-null object
width       12664 non-null int64
height      12664 non-null int64
class       12664 non-null object
xmin        12664 non-null int64
ymin        12664 non-null int64
xmax        12664 non-null int64
ymax        12664 non-null int64
dtypes: int64(6), object(2)
memory usage: 791.6+ KB
None


# Step 2: add gamma corrected images

In [12]:
def adjust_gamma(image , gamma = 1.0):
    invgamma = 1 / gamma
    table = np.array([ ((i / 255.0 )**invgamma)*255 for i in np.arange(0,256)]).astype("uint8")
    return cv2.LUT(image, table)

def get_image(input_img, xml):
    img = os.path.join(image_path,input_img)
    filename = img.split("/")[-1]
    filename = filename.split(".")[0]
    xml_path = os.path.join(image_path, xml)
    original = cv2.imread(img,1)
    gamma_decrease = round(np.random.random()*0.2 + 0.3, 1)
    gamma_increase = round(np.random.random() + 2.5, 1)
    for gamma in [gamma_decrease, gamma_increase]:
        if gamma < 1:
            direction = 'down'
        else:
            direction = 'up'
        adjusted = adjust_gamma(original, gamma=gamma)
        #cv2.putText(adjusted, "g={}".format(round(gamma,1)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
        cv2.imwrite('{}/{}_gamma_{}.jpg'.format(image_path,filename,direction), adjusted)
    return

In [13]:
files = os.listdir(image_path)
images = [img.split(".")[0] for img in files if img[-3:] == 'jpg']
length = len(images)
file_pair = [(img+".jpg",img+".xml") for img in images]
for pair in file_pair:
    get_image(pair[0],pair[1])

# Step 3: create gamma file data and add to csv

In [13]:
def gamma_to_csv(csv):
    df = pd.read_csv(csv)
    down_df = df.copy()
    up_df = df.copy()
    down_df['filename'] = down_df['filename'].apply(lambda f: '{}_gamma_down.jpg'.format(f.split(".")[0]))
    up_df['filename'] = up_df['filename'].apply(lambda f: '{}_gamma_up.jpg'.format(f.split(".")[0]))
    gamma_df = pd.concat([df, down_df, up_df])
    gamma_df.to_csv("data/gamma_labels.csv",index=False)
    return gamma_df

In [14]:
gamma_df = gamma_to_csv(os.path.join(data_path,'labels.csv'))
gamma_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37992 entries, 0 to 12663
Data columns (total 8 columns):
filename    37992 non-null object
width       37992 non-null int64
height      37992 non-null int64
class       37992 non-null object
xmin        37992 non-null int64
ymin        37992 non-null int64
xmax        37992 non-null int64
ymax        37992 non-null int64
dtypes: int64(6), object(2)
memory usage: 2.6+ MB


# Step 4: create images with horizontal flip

In [17]:
def images_flip_horizontal():
    files = os.listdir('images')
    files = [f for f in files if f[-3:] == 'jpg']
    for image in files:
        img = cv2.imread("images/{}".format(image),1)
        hflip = cv2.flip(img,1)
        cv2.imwrite('images/{}_flip.jpg'.format(image.split(".")[0]),hflip)
    return

In [18]:
images_flip_horizontal()

# Step 5: create labels for flipped images and add to csv

In [49]:
def horizontal_csv(csv):
    image_data = pd.read_csv(csv)
    hflip = image_data.copy()

    hflip['filename'] = hflip['filename'].apply(lambda f: "{}_flip.jpg".format(f.split(".")[0]))

    new_xmin = hflip['xmax'].apply(lambda x: 640 - x)
    new_xmax = hflip['xmin'].apply(lambda x: 640 - x)

    hflip['xmin'] = new_xmin
    hflip['xmax'] = new_xmax

    hflip.to_csv('data/flipped_data.csv',index=False)
    all_data = pd.concat([image_data,hflip])
    all_data.to_csv("data/all_data.csv",index=False)

    return hflip,all_data

In [50]:
hflip,all_data = horizontal_csv('data/gamma_labels.csv')

# Step 6: train/test split

In [89]:
def train_test_split(data_csv):
    df = pd.read_csv(data_csv)
    all_files = [f for f in os.listdir(image_path) if f[-3:] == 'jpg']
    split_loc = int(len(all_files) * .9)
    randomize = np.random.permutation(all_files)
    
    os.mkdir(os.path.join(image_path,'train'))
    os.mkdir(os.path.join(image_path,'test'))

    for filename in all_files[:split_loc]:
        shutil.copy(os.path.join(image_path,filename), 
                    os.path.join(image_path,'train',filename))
    for filename in all_files[split_loc:]:
        shutil.copy(os.path.join(image_path,filename), 
                    os.path.join(image_path,'test',filename))
        
    train_files = os.listdir(os.path.join(image_path,'train'))
    test_files = os.listdir(os.path.join(image_path,'test'))

    train = df[df['filename'].isin(train_files)]
    test = df[df['filename'].isin(test_files)]
    train.to_csv("data/train_labels.csv",index=False)
    test.to_csv("data/test_labels.csv",index=False)
    return train, test

In [90]:
train,test = train_test_split('data/all_data.csv')

# Step 7: generate TFRecords
### run generate_tfrecord.py script from command line using commands below
python generate_tfrecord.py --csv_input=data/train_labels.csv --output_path=data/train.record<br>
python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=data/test.record
